In [1]:
import pandas as pd
import numpy as np

In [2]:
# чтение датасета торгов
data = pd.read_excel('cleared_data.xlsx')
data.head()

,Заказчик,ИНН_заказчик,Стоимость\n(руб.),Сфера деятельности,Наименование публикации,Регион поставки,Город поставки,Поставщик,ИНН_поставщик,Победитель,Статус допуска,Форма публикации,Тип торгов,Торговая площадка,Ссылка на источник
0,УПРАВЛЕНИЕ РОСПОТРЕБНАДЗОРА ПО СТАВРОПОЛЬСКОМУ...,2.636045e+09,2000000.0,"[ОКПД2 27.20] Батареи и аккумуляторы, [ОКПД2 4...","Оказание услуг по диагностике, техническому об...",Ставропольский край,Ставрополь,"ООО ""ГЕВИННЕР""",2.635830e+09,NaN,NaN,Контракт,Аукцион электронный,ЕИС,Госзакупки 44ФЗ/94ФЗ
1,"ГБУ РО ""ЦРБ"" В НЕКЛИНОВСКОМ РАЙ",6.123004e+09,6710121.0,"[ОКПД2 29.10] Средства автотранспортные, [КТРУ...","Закупка автомобилей для ГБУ РО ""ЦРБ"" в Неклино...",Ростовская область,Неклиновский район,"ООО ""БНМ-3""",3.241504e+09,NaN,NaN,Контракт,Аукцион электронный,ЕИС,Госзакупки 44ФЗ/94ФЗ
2,"МКУ ""СЛУЖБА ЗАКАЗЧИКА""",1.001033e+09,68000.0,[ОКПД2 29.32] Комплектующие и принадлежности д...,Поставка каналопромывочных насадок,Республика Карелия,Петрозаводск,Иванов Федор Николаевич,2.129141e+11,Победитель,Неизвестно,Торговая процедура,Мелкая закупка,Электронный магазин г.о. Петрозаводск,Электронный магазин г.о. Петрозаводск 44-ФЗ
3,"ООО ""СМУ-12 МОСМЕТРОСТРОЯ""",7.718925e+09,111552.0,[ОКПД2 29.32] Комплектующие и принадлежности д...,ЗАПАСНЫЕ ЧАСТИ ДЛЯ ТРАНСПОРТНЫХ СРЕДСТВ,Москва,Москва,Краснова Дина Анатольевна,2.107023e+11,Победитель,Неизвестно,Торговая процедура,Запрос котировок,Портал поставщиков Москвы,ПОРТАЛ ПОСТАВЩИКОВ МОСКВЫ 223-ФЗ
4,"ГБУ ДО ""САШ""",7.715364e+09,39400.0,[ОКПД2 29.20] Кузова (корпуса) для автотранспо...,"Мусорный контейнер GRATAN БК-0,8 УШ на большег...",Москва,Москва,"ООО ""ГРУППА ГРАТАН""",7.719176e+09,Победитель,Неизвестно,Торговая процедура,Запрос котировок,Портал поставщиков Москвы,ПОРТАЛ ПОСТАВЩИКОВ МОСКВЫ 44-ФЗ


In [3]:
data.shape

(14067, 15)

In [4]:
# удаление торгов, в которых не указано цена
data = data[data['Стоимость\n(руб.)'].isnull() == False]

In [6]:
# удаление записей, в которых 2 раза присутствует название региона
fixed_region = data[data['Регион поставки'].str.contains(';') == True]['Регион поставки'].apply(lambda x:x.split(';')[0])
data.loc[data['Регион поставки'].str.contains(';')==True , 'Регион поставки'] = fixed_region

In [8]:
# конвертирование категориальной переменной 'Регион поставки' в числовые значения
from sklearn import preprocessing 
region_encoder = preprocessing.LabelEncoder() 
region_encoder.fit(data['Регион поставки'])

LabelEncoder()

In [9]:
# выбор поставщиков, у которых больше 10 успешных торгов
suppliers = pd.Series(data['ИНН_поставщик'].value_counts())
prep = list(suppliers[suppliers > 9].index)
data = data[data['ИНН_поставщик'].isin(prep)]

In [10]:
region_res = region_encoder.fit_transform(data['Регион поставки']) 

In [11]:
region_encoder.classes_ = region_encoder.classes_[:-1]
region_encoder.classes_

array(['Алтайский край', 'Амурская область', 'Архангельская область',
       'Астраханская область', 'Байконур', 'Белгородская область',
       'Брянская область', 'Владимирская область',
       'Волгоградская область', 'Вологодская область',
       'Воронежская область', 'Еврейская автономная область',
       'Забайкальский край', 'Ивановская область', 'Иркутская область',
       'Кабардино-Балкарская Республика', 'Калининградская область',
       'Калужская область', 'Камчатский край',
       'Карачаево-Черкесская Республика', 'Кемеровская область - Кузбасс',
       'Кировская область', 'Костромская область', 'Краснодарский край',
       'Красноярский край', 'Курганская область', 'Курская область',
       'Ленинградская область', 'Липецкая область', 'Магаданская область',
       'Москва', 'Московская область', 'Мурманская область',
       'Нижегородская область', 'Новгородская область',
       'Новосибирская область', 'Омская область', 'Оренбургская область',
       'Орловская област

In [14]:
# функция для извлечения кода ОКПД2
def get_okpd_code(s):
    try:
        arr = s.split()
        first = arr[0]
        code = arr[1][:-1]
        if first == '[ОКПД2' and code.split('.')[0] == '29':
            return code
    except:
        return None
get_okpd_code("[ОКПД2 29.32] Комплектующие и принадлежности для автотранспортных средств прочие")

'29.32'

In [15]:
# функция для извлечение первой цифры после точки
def level_1(arr):
    if len(arr) == 1:
        return int(arr[0].split(".")[1][0])
    else:
        res = [int(i.split(".")[1][0]) for i in arr]
        return list(set(res))

In [16]:
# функция для извлечение второй цифры после точки
def level_2(arr):
    if len(arr) == 1:
        elem = arr[0].split(".")[1][1]
        if elem == '1' or elem == '2':
            return int(elem)
        else:
            return 0
    else:
        res = []
        for i in arr:
            if i.split(".")[1][1] == '1' or i.split(".")[1][1] == '2':
                res.append(int(i.split(".")[1][1]))
        return list(set(res))

In [17]:
# конвертирование столбца 'Сфера деятельности' в числовое значение через метод One Hot Encoding
level_1_1 = []
level_1_2 = []
level_1_3 = []
level_2_1 = []
level_2_2 = []
product = list(data['Сфера деятельности'])
for i in product:
    prep =  i.split(',')
    arr1 = []
    for j in range(len(prep)):
        if get_okpd_code(prep[j]) != None:
            arr1.append(get_okpd_code(prep[j]))
    res1 = level_1(arr1)
    if type(res1) == list:
        if 1 in res1:
            level_1_1.append(1)
        else:
            level_1_1.append(0)
        if 2 in res1:
            level_1_2.append(1)
        else:
            level_1_2.append(0)
        if 3 in res1:
            level_1_3.append(1)
        else:
            level_1_3.append(0)
    else:
        if res1 == 1:
            level_1_1.append(1)
        else:
            level_1_1.append(0)
        if res1 == 2:
            level_1_2.append(1)
        else:
            level_1_2.append(0)
        if res1 == 3:
            level_1_3.append(1)
        else:
            level_1_3.append(0)
    
    res2 = level_2(arr1)
    if type(res2) == list:
        if 1 in res2:
            level_2_1.append(1)
        else:
            level_2_1.append(0)
        if 2 in res2:
            level_2_2.append(1)
        else:
            level_2_2.append(0)
    else:
        if res2 == 1:
            level_2_1.append(1)
        else:
            level_2_1.append(0)
        if res2 == 2:
            level_2_2.append(1)
        else:
            level_2_2.append(0)

In [18]:
# конвертирование столбца 'Ссылка на источник' в числовое значение через метод One Hot Encoding
col = list(data['Ссылка на источник'])
is_44 = []
for i in col:
    if '44' in i:
        is_44.append(1)
    else:
        is_44.append(0)

In [43]:
# сохранение кодировки регионов в словарь
region_encoder_json = {}
for index, value in enumerate(region_encoder.classes_):
    region_encoder_json[value] = index

In [21]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\abuzu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [22]:
# конвертирования описания тендера в 20d вектор
from gensim.models.doc2vec import Doc2Vec,\
    TaggedDocument
from nltk.tokenize import word_tokenize
 
# define a list of documents.
info = list(data['Наименование публикации'])
 
# preproces the documents, and create TaggedDocuments
tagged_data = [TaggedDocument(words=word_tokenize(doc.lower()),
                              tags=[str(i)]) for i,
               doc in enumerate(info)]
 
# train the Doc2vec model
model = Doc2Vec(vector_size=20,
                min_count=2, epochs=50)
model.build_vocab(tagged_data)
model.train(tagged_data,
            total_examples=model.corpus_count,
            epochs=model.epochs)
 
# get the document vectors
document_vectors = [model.infer_vector(
    word_tokenize(doc.lower())) for doc in info]
 


In [34]:
# сохранение модели 
import pickle
filename = 'description_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [23]:
# создание датасета с данными для обучения модели
train_data = pd.DataFrame()
train_data['cost'] = data['Стоимость\n(руб.)']
train_data['level_1_1'] = level_1_1
train_data['level_1_2'] = level_1_2
train_data['level_1_3'] = level_1_3
train_data['level_2_1'] = level_2_1
train_data['level_2_2'] = level_2_2
train_data['region'] = list(region_res)
train_data['is_44'] = is_44

In [24]:
# фунция для извлечения столбца из вектора
def get_column(data, i):
    res = []
    for row in range(len(data)):
        res.append(data[row][i])
    return res

In [25]:
for i in range(20):
    train_data[f"title_vector_{i}"] = get_column(document_vectors, i)

In [26]:
supplier_encoder = preprocessing.LabelEncoder() 
train_data['supplier_id'] = supplier_encoder.fit_transform(data['ИНН_поставщик'])

In [38]:
train_data.head()

,cost,level_1_1,level_1_2,level_1_3,level_2_1,level_2_2,region,is_44,title_vector_0,title_vector_1,...,title_vector_11,title_vector_12,title_vector_13,title_vector_14,title_vector_15,title_vector_16,title_vector_17,title_vector_18,title_vector_19,supplier_id
1,6710121.0,1,0,0,0,0,60,1,0.252373,-0.539630,...,-0.442900,0.162875,-0.014480,0.349676,0.382625,-0.145744,-0.067603,-0.418531,-0.409048,44
15,5395.0,0,0,1,0,1,51,1,-0.032253,0.046215,...,0.072308,0.155255,0.326737,0.006790,0.103095,0.146914,-0.070768,-0.172116,-0.065099,9
22,34330.0,0,0,1,0,1,9,0,-0.033844,0.004366,...,-0.167090,-0.000514,-0.281304,0.014696,0.088821,0.396348,-0.107552,-0.244011,-0.093022,52
26,1200000.0,1,0,0,0,0,46,1,0.107452,0.115119,...,-0.170172,0.253630,-0.131301,0.033169,0.276916,0.403258,-0.050170,-0.184235,-0.236303,5
27,1144000.0,1,0,0,0,0,45,1,-0.054130,0.034699,...,-0.019941,0.213211,-0.100584,0.058279,0.060351,0.286228,-0.022537,-0.199132,-0.115796,143


In [41]:
# сохранение кодировки поставщиков в словарь
suppliers_encoder_json = {}
for index, value in enumerate(supplier_encoder.classes_):
    suppliers_encoder_json[int(value)] = index

In [44]:
#сохранение словарей в json

import json
with open("supplier_encoder.json", "w") as outfile: 
    json.dump(suppliers_encoder_json, outfile)
    
with open("region_encoder.json", "w") as outfile: 
    json.dump(region_encoder_json, outfile)

In [61]:
# сохранение данных для обучения модели в csv
train_data.to_csv('train_data.csv')